In [122]:
import pandas as pd

df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

not_fit = True


In [2]:
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
nltk.download('stopwords')
stop = stopwords.words('english')

def tokenizer_stem_nostop(text):
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]

[nltk_data] Downloading package stopwords to /Users/AtSu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import numpy as np
from bs4 import BeautifulSoup
from collections import defaultdict
import src.proc_data as procd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
"""
train_channel = np.zeros((df.values.shape[0], 4), dtype=object)
test_channel = np.zeros((df_test.values.shape[0], 4), dtype=object)
columns = ['entertainment', 'world', 'social-media', 'tech']


for i in range(df['Page content'].values.shape[0]):
    channel = procd.parse_channel(df['Page content'][i])
    if channel in columns:
        train_channel[i, columns.index(channel)] = 1

pd.DataFrame(data=train_channel, columns=columns).to_csv('train_channel.csv', index=False)
        
for i in range(df_test['Page content'].values.shape[0]):
    channel = procd.parse_channel(df_test['Page content'][i])
    if channel in columns:
        test_channel[i, columns.index(channel)] = 1
        
pd.DataFrame(data=test_channel, columns=columns).to_csv('test_channel.csv', index=False)
"""
"""
train_parse = np.zeros((df.values.shape[0], 5),dtype=object)

for i in range(df['Page content'].values.shape[0]):
#for i in range(5):

    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3]\
            = procd.parse_html(df['Page content'][i])
    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))

pd.DataFrame(data=train_parse, columns=['title', 'content', 'author', 'n_fig', 'n_words']).to_csv('train_b_parsed.csv')

train_parse = np.zeros((df_test.values.shape[0], 5),dtype=object)

#for i in range(5):

for i in range(df_test['Page content'].values.shape[0]):
    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3] \
            = procd.parse_html(df_test['Page content'][i])
    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))

pd.DataFrame(data=train_parse, columns=['title', 'content', 'author','n_fig', 'n_words']).to_csv('test_b_parsed.csv')
"""

"\ntrain_parse = np.zeros((df.values.shape[0], 5),dtype=object)\n\nfor i in range(df['Page content'].values.shape[0]):\n#for i in range(5):\n\n    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3]            = procd.parse_html(df['Page content'][i])\n    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))\n\npd.DataFrame(data=train_parse, columns=['title', 'content', 'author', 'n_fig', 'n_words']).to_csv('train_b_parsed.csv')\n\ntrain_parse = np.zeros((df_test.values.shape[0], 5),dtype=object)\n\n#for i in range(5):\n\nfor i in range(df_test['Page content'].values.shape[0]):\n    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3]             = procd.parse_html(df_test['Page content'][i])\n    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))\n\npd.DataFrame(data=train_parse, columns=['title', 'content', 'author','n_fig', 'n_words']).to_csv('test_b_parsed.csv')\n"

In [125]:
df = pd.read_csv('train_parsed.csv')
df_test = pd.read_csv('test_parsed.csv')


In [77]:
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from time import time

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()
    

n_topics = 5
n_features = 2**12
n_top_words = 20


t0 = time()
tf = CountVectorizer(max_df=0.95, min_df=2,
                        max_features=n_features,
                        tokenizer=tokenizer_stem_nostop)
bow = tf.fit_transform(df['content'])
print('finish fitting %.2f' %(time()-t0))

lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
bow = lda.fit_transform(bow)

tf_feature_names = tf.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

finish fitting 145.95
Topic #0:
compani facebook user new social time peopl work like say year busi use media make market million one job ad
Topic #1:
said space year research use one could new also imag time like system would flight studi accord water say human
Topic #2:
app appl use devic new also phone googl imag one iphon user like featur design android make screen compani smartphon
Topic #3:
said twitter report state govern pic com u polic peopl presid accord would citi also one year offic countri protest
Topic #4:
imag one video like game time show year get world new first make see play day look also go fan



In [89]:
print(bow.shape)
print(bow)

(27643, 5)
[[  3.50795959e-02   8.33542922e-01   6.99043277e-04   1.08507109e-01
    2.21713300e-02]
 [  1.86605444e-01   1.32445538e-03   7.69332634e-01   4.14141028e-02
    1.32336407e-03]
 [  1.18914680e-01   1.45721425e-03   1.46798102e-03   7.28015183e-02
    8.05358606e-01]
 ..., 
 [  5.62961070e-03   5.76766089e-03   5.62673829e-03   5.62972694e-03
    9.77346263e-01]
 [  9.96501419e-01   8.72555785e-04   8.74678630e-04   8.79219619e-04
    8.72126813e-04]
 [  2.54645898e-01   1.59345679e-01   8.49558459e-04   8.43194316e-04
    5.84315670e-01]]


In [146]:
from scipy.sparse import hstack
from sklearn.preprocessing import PolynomialFeatures
train_weekday = pd.read_csv('train_weekday.csv').values
train_time = pd.read_csv('train_time.csv').values
train_channel = pd.read_csv('train_channel.csv').values
poly = PolynomialFeatures(degree=3)

#X_train_aug = np.concatenate((train_channel, train_weekday, train_time, bow, df['n_fig'].reshape(-1,1)), axis=1)
X_train_aug = np.concatenate((train_channel, train_weekday, train_time, df['n_fig'].reshape(-1,1)), axis=1)
X_train_aug = poly.fit_transform(X_train_aug)[:, 1:]

print(X_train_aug.shape)

(27643, 559)


In [147]:
from sklearn.model_selection import train_test_split
# hold out testing set
y_train = df['Popularity']
X_htrain, X_htest, y_htrain, y_htest = train_test_split(X_train_aug, y_train, test_size=0.3, random_state=0)


In [148]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
SGD = LogisticRegression(C=3.0, random_state=0)
#SGD = MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

SGD.fit(X_htrain, y_htrain)

pred = SGD.predict(X_htrain)
pred_t = SGD.predict(X_htest)

score_train = metrics.accuracy_score(y_htrain, pred)
score_test = metrics.accuracy_score(y_htest, pred_t)

print("SGD score (train): %f, (test): %f"%(score_train, score_test))
ada = AdaBoostClassifier(base_estimator=SGD, algorithm='SAMME', n_estimators=50)
ada = ada.fit(X_htrain, y_htrain)
pred = ada.predict(X_htrain)
pred_t = ada.predict(X_htest)

score_train = metrics.accuracy_score(y_htrain, pred)
score_test = metrics.accuracy_score(y_htest, pred_t)

print("SGD score (train): %f, (test): %f"%(score_train, score_test))
SGD.fit(X_train_aug, y_train)

pred = SGD.predict(X_htrain)
pred_t = SGD.predict(X_htest)
score_train = metrics.accuracy_score(y_htrain, pred)
score_test = metrics.accuracy_score(y_htest, pred_t)

# best case here
print("SGD score (train): %f, (test): %f"%(score_train, score_test))
candidates = []
candidates.append({'clf':SGD, 'C':1, 'score':1.0})

ada = AdaBoostClassifier(base_estimator=SGD, algorithm='SAMME', n_estimators=50)
ada = ada.fit(X_train_aug, y_train)
pred = ada.predict(X_htrain)
pred_t = ada.predict(X_htest)

score_train = metrics.accuracy_score(y_htrain, pred)
score_test = metrics.accuracy_score(y_htest, pred_t)

print("SGD score (train): %f, (test): %f"%(score_train, score_test))

candidates.append({'clf' : ada, 'C' : 0.5, 'score':0.55679})

SGD score (train): 0.550388, (test): 0.546123
SGD score (train): 0.546408, (test): 0.538526
SGD score (train): 0.550543, (test): 0.557699
SGD score (train): 0.547080, (test): 0.552394


In [86]:
test_weekday = pd.read_csv('test_weekday.csv').values
test_time = pd.read_csv('test_time.csv').values
test_channel = pd.read_csv('test_channel.csv').values

"""
bow_test = tf.transform(df_test['content'])
bow_test = lda.transform(bow_test)
test_weekday = poly.transform(test_weekday)[:,1:]

X_test_con = np.concatenate((test_channel, test_weekday, test_time, bow_test, df_test['n_fig'].reshape(-1,1)), axis=1)
"""
#X_test_con = np.concatenate((test_weekday, test_time), axis=1)
#X_test_con = test_weekday * test_time
#print(X_test_con.shape)
#X_test_std = sc_x.fit_transform(X_test_con)

In [151]:

X_test_con = np.concatenate((test_channel, test_weekday, test_time, df_test['n_fig'].reshape(-1,1)), axis=1)
X_test_con = poly.transform(X_test_con)[:,1:]

In [152]:

for cand in candidates:
    predict = cand['clf'].predict(X_test_con)
    output = np.zeros((X_test_con.shape[0], 2), dtype=int)
    output[:, 0] = df_test['Id']
    output[:, 1] = predict
    df_output = pd.DataFrame(data=output, columns=['Id', 'Popularity'])
    df_output.to_csv('test___%f.csv' % (cand['C']), index=False)